## Predicting SP&P 500 Index based on historic data
The goal of this project is to predict S&P 500 Index based on historical data. The idea is to create new indicators based on given data that does not induct future knowledge into our model. Because the stock price of each day can affect the volumes traded and price of the next significantly it is important to use rolling averages like a 5 day, 30 day and 1 year rolling average of closing price.

In [1]:
#importing relevant libraries
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.linear_model import LinearRegression

In [3]:
df=pd.read_csv('sphist.csv')
df['Date']=pd.to_datetime(df['Date'], infer_datetime_format=True)
df=df.sort_values(by='Date',ascending=True)

In [5]:
## Calculating new additional indicators based on 5,30 and 365 day rolling windows. 
## Current days data should be excluded when calculating rolling averages.
df_test = df.copy(deep=True)
df_test['day_5']=0
df_test['day_30']=0
df_test['day_yr']=0
df_test['avg_5day_volume']=0
df_test['avg_yr_volume']=0
for index, row in df_test.iterrows():
    current_date=df_test.iloc[index]['Date']
    current_date_10 = current_date-timedelta(days=15)
    current_date_30 = current_date-timedelta(days=65)
    current_date_365 = current_date-timedelta(days=660)
    last_5_dates=df_test[(df_test['Date']>current_date_10) & (df_test['Date']<current_date)].tail(5)
    last_30_dates=df_test[(df_test['Date']>current_date_30) & (df_test['Date']<current_date)].tail(30)
    last_365_dates=df_test[(df_test['Date']>current_date_365) & (df_test['Date']<current_date)].tail(365)
    if (int(len(last_5_dates))==5):
        five_day_avg=last_5_dates['Close'].mean()
        fiveday_volume=last_5_dates['Volume'].mean()
        df_test.day_5.iloc[index]=five_day_avg
        df_test.avg_5day_volume.iloc[index]=fiveday_volume
    if (int(len(last_30_dates))==30):
        thirty_day_avg=last_5_dates['Close'].mean()
        df_test.day_30.iloc[index]=thirty_day_avg
    if (int(len(last_365_dates))==365):
        yr_avg=last_365_dates['Close'].mean()
        yr_volume=last_365_dates['Volume'].mean()
        df_test.day_yr.iloc[index]=yr_avg
        df_test.avg_yr_volume.iloc[index]=yr_volume

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
df_test.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_30,day_yr,avg_5day_volume,avg_yr_volume
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,0.000,0.0,0.0,0.0,0.0
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,0.000,0.0,0.0,0.0,0.0
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,0.000,0.0,0.0,0.0,0.0
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,0.000,0.0,0.0,0.0,0.0
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,0.000,0.0,0.0,0.0,0.0
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,0.0,0.0,2046000.0,0.0
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,0.0,0.0,2226000.0,0.0
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,0.0,0.0,2374000.0,0.0
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,0.0,0.0,2458000.0,0.0
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,0.0,0.0,2722000.0,0.0


In [8]:
#Selecting only relevant data which has values and rows which do have no nulls
df_final = df_test[df_test['Date']> datetime(year=1951, month=6, day=18)]       
df_final = df_final.dropna(axis=0)
train=df_final[df_final['Date']<datetime(year=2013, month=1, day=1)]
test=df_final[df_final['Date']>=datetime(year=2013, month=1, day=1)]   

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 739 entries, 738 to 0
Data columns (total 12 columns):
Date               739 non-null datetime64[ns]
Open               739 non-null float64
High               739 non-null float64
Low                739 non-null float64
Close              739 non-null float64
Volume             739 non-null float64
Adj Close          739 non-null float64
day_5              739 non-null float64
day_30             739 non-null float64
day_yr             739 non-null float64
avg_5day_volume    739 non-null float64
avg_yr_volume      739 non-null float64
dtypes: datetime64[ns](1), float64(11)
memory usage: 75.1 KB


In [14]:
# Creating model with 3 features
lr_model1 = LinearRegression()
features_model1=['day_5','day_30','day_yr']
lr_model1.fit(train[features_model1],train['Close'])
predict_model1 = lr_model1.predict(test[features_model1])

mae_model1 = mean_absolute_error(predict_model1,test['Close'])
mse_model1 = mean_squared_error(predict_model1,test['Close'])
            
print("MAE for model with 3 features is {:.2f}".format(mae_model1))
print("MSE for model with 3 features is {:.2f}".format(mse_model1))

MAE for model with 3 features is 16.13
MSE for model with 3 features is 491.88


In [15]:
# Creating model with 5 features
lr_model2 = LinearRegression()
features_model2=['day_5','day_30','day_yr','avg_5day_volume','avg_yr_volume']
lr_model2.fit(train[features_model2],train['Close'])
predict_model2 = lr_model2.predict(test[features_model2])

mae_model2 = mean_absolute_error(predict_model2,test['Close'])
mse_model2 = mean_squared_error(predict_model2,test['Close'])
            
print("MAE for model with 3 features is {:.2f}".format(mae_model2))
print("MSE for model with 3 features is {:.2f}".format(mse_model2))

MAE for model with 3 features is 16.11
MSE for model with 3 features is 492.88
